In [ ]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
def load_and_preprocess_image(img_path):
    image = np.array(Image.open(img_path).resize((224, 224))) / 255.0
    return image

def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    return lr

In [ ]:
tops_images = ["Womens Surrealistic Themes Funny Summer Plus Size T-Shirts - 2 _ L.jpg", "Mens Astronaut Shooting Cartoon Print Crew Neck Short Sleeve T-Shirts - Khaki _ XL.jpg",
               "T-shirt Women Funny Cartoon Short Sleeve Streetwear - FuD 4127 bai _ M.jpg", "Breathable Solid Color round Neck Short Sleeve Cotton T-Shirts.jpg",
               "Vossiy Graphic Tees Men Fashion Print T-Shirts Unisex Cotton Tee Hipster Casual Streetwear Loose Summer Tee Top.jpg"]

bottoms_images = ["download (1).jpg","Women's Designer Denim _ Moda Operandi.jpg","Designer Wide-Leg Jeans for Women.jpg","Jacquemus Fashion Collections For Women _ Moda Operandi.jpg",
                 "download.jpg"]

In [ ]:
labels = [0] * len(tops_images) + [1] * len(bottoms_images)
images_train, images_test, labels_train, labels_test = train_test_split(
    tops_images + bottoms_images, labels, test_size=0.2, random_state=42
)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
images_train = np.array([load_and_preprocess_image(img) for img in images_train])
labels_train = np.array(labels_train, dtype=np.float32)

In [ ]:
augmented_train_data = []
for img, label in zip(images_train, labels_train):
    img_array = np.expand_dims(img, axis=0)
    augmented_images = [datagen.random_transform(img_array[0]) for _ in range(32)]
    augmented_train_data.extend([(augmented_image, label) for augmented_image in augmented_images])

In [ ]:
augmented_train_data = np.array(augmented_train_data, dtype=object)
images_train_augmented, labels_train_augmented = np.stack(augmented_train_data[:, 0]), augmented_train_data[:, 1]
images_train_augmented = images_train_augmented.reshape(-1, 224, 224, 3)

In [ ]:
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the pre-trained weights

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
labels_train_augmented = np.array(labels_train_augmented, dtype=np.float32)
labels_train_augmented = np.array(labels_train_augmented, dtype=int)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    LearningRateScheduler(lr_schedule)
]

In [ ]:
history = model.fit(images_train_augmented, labels_train_augmented, 
                    epochs=50, batch_size=32, 
                    validation_split=0.2, callbacks=callbacks)

In [ ]:
images_test = np.array([load_and_preprocess_image(img) for img in images_test])
labels_test = np.array(labels_test, dtype=np.float32)

In [ ]:
test_loss, test_accuracy = model.evaluate(images_test, labels_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")